# Flow Counter

* This notebook is used to count flows, in respect to each geochem group, from different wells.
-----------------------------------------------------------------------------------------------------------------   
* Main parts include:
     * A flow counter algorithm
     * A function generalizing the flow-counter algorithm -- TODO

## Importing libraries and data

In [109]:
# Importing libraries
import pandas as pd
import re

# Reading .csv
data = pd.read_csv(r'Data/CSV/1-AV-001-PR.csv', sep=';') ## Dataset1
# data = pd.read_csv(r'Data/CSV/1-CS-002-PR.csv', sep=';') ##Dataset2
data.head()

,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion
0,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"11,2","31,36","0,589285714"
1,1-AV-001-PR,Paranapanema,Simple lava (basic | massive interior),"18,48",NaN,NaN
2,1-AV-001-PR,Paranapanema,Simple lava (basic | L. crust),"1,68",NaN,NaN
3,1-AV-001-PR,Paranapanema,Siliciclastics,"1,23","1,23",NaN
4,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"29,81","88,48","0,643196203"


## Flow Counter Algorithm/Function
* This code is used to run throught the WellCad data and counting the # flows per geochemical group.
-----------------------------------------------------------------------------------------------------------------   
* Things considered to indicate flow "change":
    * Upper crust of lava flow
    * Compound flows are counted as singular flow
    * Interbeds indicate flow changes
    * Unit changes indicate flow changes
-----------------------------------------------------------------------------------------------------------------    
* Output:
    * A flow_list list which contains the exact same number of values as the input Pandas DataFrame. 
    * This list can then be appended to the given dataframe to generate a flow# column.

In [113]:
def flow_counter(data, flow_list, flow_dict):
    ''
    
    # variables
    flow_counter = 0
    
    # Algorithm
    for i in range(len(data)):
        if i==0: # Setting up the first iteration
            flow_counter += 1
            flow_list.append(flow_counter)
        elif i == (len(data)-1): # Checking for the end of the list (working around out-of-bounds bug)
            if re.search(r'compound', data['Facies'].iloc[i], re.IGNORECASE) != None:
                
            
            else:
                flow_list.append(flow_counter)
                flow_dict[data['Unit'].iloc[i]] = flow_counter
            break      
        elif re.search(r'lava', data['Facies'].iloc[i]) != None: # If we're with a 'lava' facies
            if data['Unit'].iloc[i] == data['Unit'].iloc[i-1]: # If unit is the same as before
                if data['Facies'].iloc[i] == 'Simple lava (basic | U. crust)' or data['Facies'].iloc[i] == 'Compound lava (basic)': # Check for Simple lower crust OR compound flows:
                    flow_counter += 1
                    flow_list.append(flow_counter)
                    flow_dict[data['Unit'].iloc[i]] = flow_counter
                else:
                    flow_list.append(flow_counter)
                    #flow_dict[data['Unit'].iloc[i]] = flow_counter
            else: # If Unit changes
                if data['Unit'].iloc[i] in flow_dict: # If unit has already been added as a key to the flow_dict
                    flow_dict[data['Unit'].iloc[i-1]] = flow_counter
                    flow_counter = flow_dict[data['Unit'].iloc[i]]
                    flow_list.append(flow_counter)
                else: # If unit is NOT already associated with a value in the flow_dict
                    flow_dict[data['Unit'].iloc[i-1]] = flow_counter
                    flow_counter = 1
                    flow_dict[data['Unit'].iloc[i]] = flow_counter
                    flow_list.append(flow_counter)
                    # Check if the facies is an upper crust/compound lava
                    #if data['Facies'].iloc[i]  == 'Compound lava (basic)': # Working around the situtation where the unit changes into a "compound lava" facies.                   
                        #flow_counter += 1
                    #else:
                        #continue
        else: # Check for siliciclastic OR volcaniclastic; append a "-" for these units:
            if data['Unit'].iloc[i] == data['Unit'].iloc[i-1]: # If unit is the same as before
                flow_list.append('-')
            else:
                flow_list.append('-')
                flow_counter = 1

In [114]:
## Function test
cs_pr_dict = {}
cs_pr_list = []
flow_counter(data,cs_pr_list,cs_pr_dict)
print(cs_pr_list)
print(cs_pr_dict)
data['flow#'] = cs_pr_list
data.head(83)

[1, 1, 1, '-', 2, 2, 2, '-', 3, 3, 3, '-', 4, 4, 4, 5, '-', 6, 7, 7, 7, 8, 9, 9, 9, '-', 10, 10, 10, '-', 11, 11, 11, '-', 12, 12, 12, '-', 13, 13, 13, 13, 13, 13, '-', 14, 14, 14, 14, 15, 15, 15, 15, '-', 16, 16, 16, '-', 16, 16, 16, 17, '-', 18, 18, 18, '-', 19, 20, 20, 21, '-', 22, 22, 22, 23, 23, 23, '-', 24, 24, 24, 1, 1, 1, 2, 2, 2, 2, 3, 24, 24, 24, '-', 25, 26, 26, 26, 27, 28, 28, 28, '-', 3, '-', 4, 5, 5, 5, '-', 6, 7, 7, 7, 8, 9, 10, 10, 10, '-', 11, 11, 11, '-', 12, 12, 12, 13, 13, 13, '-', 14, 15, 15, 15, 15, 15, 15, 16, '-', 17, 17, 17, 18, 18, 18, 19, 19, 19, '-', 20, 20, 20, 21, 22, 22, 22, '-', 23, 23, 23, 24, 24, 24, '-', 25, 26, 26, 26, 27, 27, 27, '-', 28, 28, 28, 29, 29, 29, 29, '-', 30, 30, 30, 1, 2, 2, 2, 3, 4, 4, 4, 4]
{'Paranapanema': 28, 'Pitanga': 30, 'Vale do Sol': 4}


,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion,flow#
0,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"11,2","31,36","0,589285714",1
1,1-AV-001-PR,Paranapanema,Simple lava (basic | massive interior),"18,48",NaN,NaN,1
2,1-AV-001-PR,Paranapanema,Simple lava (basic | L. crust),"1,68",NaN,NaN,1
3,1-AV-001-PR,Paranapanema,Siliciclastics,"1,23","1,23",NaN,-
4,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"29,81","88,48","0,643196203",2
...,...,...,...,...,...,...,...
78,1-AV-001-PR,Paranapanema,Siliciclastics,"2,05","2,05",NaN,-
79,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"9,87","23,15","0,525269978",24
80,1-AV-001-PR,Paranapanema,Simple lava (basic | massive interior),"12,16",NaN,NaN,24
81,1-AV-001-PR,Paranapanema,Simple lava (basic | L. crust),"1,12",NaN,NaN,24


## Others /// Notes: